In [60]:
import pandas as pd
import numpy as np
import glob
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error as mae
import xgboost as xgb 
from sklearn.metrics import mean_absolute_error 
from sklearn.model_selection import train_test_split
#使用したライブラリは以上

# パスで指定したファイルの一覧をリスト形式で取得. （ここでは一階層下のtestファイル以下）
csv_files = glob.glob('./train/train/*.csv')

In [61]:


#csvファイルの中身を追加していくリストを用意
data_list = []

#読み込むファイルのリストを走査
for file in csv_files:
    data_list.append(pd.read_csv(file))

#リストを全て行方向に結合
#axis=0:行方向に結合, sort
df = pd.concat(data_list, axis=0)
df1 = df

C:\Users\shosh\AppData\Local\Temp\ipykernel_8336\1329878912.py:6: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  data_list.append(pd.read_csv(file))
C:\Users\shosh\AppData\Local\Temp\ipykernel_8336\1329878912.py:6: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  data_list.append(pd.read_csv(file))
C:\Users\shosh\AppData\Local\Temp\ipykernel_8336\1329878912.py:6: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  data_list.append(pd.read_csv(file))


In [66]:
df = df1

In [67]:


# 表示する最大列数を設定
pd.set_option('display.max_columns', 50)



quarter_conversion = {
    '年第1四半期': '.25',
    '年第2四半期': '.50',
    '年第3四半期': '.75',
    '年第4四半期': '.99'
}
def convert_to_float(date_str):
    for key, value in quarter_conversion.items():
        if key in date_str:
            year = date_str.replace(key, '')
            return float(year + value)
    return None





def data_pre(df):
    nonnull_list = []
    for col in df.columns:
        nonnull = df[col].count()
    if nonnull == 0:
        nonnull_list.append(col)

    df = df.drop(nonnull_list, axis=1)
    if 'ID' in df.columns:
        df = df.drop('ID', axis = 1)



    dis = {
      '30分～60分':45,
      '1H～1H30':75,
      '2H～':120,
      '1H30～2H':105
    }
    df['最寄駅：距離（分）'] = df['最寄駅：距離（分）'].replace(dis).astype(float)

    df['面積（㎡）'] = df['面積（㎡）'].replace('2,000㎡以上',2000).astype(float)


    df['建築年'] = df['建築年'].apply(lambda x: 1945 if x == "戦前" else (int(x[:-1]) if isinstance(x, str) else np.nan))
    # df = df.drop(columns=['建築年'])


    df['取引時点'] = df['取引時点'].apply(convert_to_float)
    
    df['面積ー築年数']= df["面積（㎡）"] - df['建築年']
    df['建築年ー取引時点']= df["建築年"] - df['取引時点']
    # df['面積ー取引時点']= df["面積（㎡）"] - df['取引時点'] #ないほうがいい
    
    for col in ["都道府県名", "地区名","市区町村名","種類", "最寄駅：名称", "間取り", "建物の構造", "用途", "今後の利用目的", "都市計画", "改装", "取引の事情等"]:
                df[col] = df[col].astype("category")
    
    # df = df.drop(columns=['土地の形状','間口','延床面積（㎡）'])
    df = df.drop(columns=['種類',"都道府県名",'地域','土地の形状','間口','延床面積（㎡）','前面道路：方位','前面道路：種類','前面道路：幅員（ｍ）'])

    return df

df = data_pre(df)

In [69]:
df_train,df_val = train_test_split(df,test_size=0.2)

col = '取引価格（総額）_log'
train_y = df_train[col]
train_x = df_train.drop(col,axis=1)

val_y = df_val[col]
val_x = df_val.drop(col,axis=1)

trains = lgb.Dataset(train_x,train_y)
valids = lgb.Dataset(val_x,val_y)
params = {
    'objective': 'regression',
    'metric': 'mae',
    'boosting_type': 'gbdt',
    'learning_rate': 0.01,
    'num_leaves': 31,
    'max_depth': -1,
    'min_data_in_leaf': 20,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'lambda_l1': 0.1,
    'lambda_l2': 0.1,
    'min_split_gain': 0.01,
    'n_jobs': -1,
    'verbosity': -1,
   
  
    'max_bin': 25 ,
     'bin_construct_sample_cnt': 200000# max_binの値を255以下に設定
}




model = lgb.train(params, trains, valid_sets=valids, num_boost_round=3000, callbacks=[lgb.early_stopping(stopping_rounds=100)])


Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[3000]	valid_0's l1: 0.0780838


In [65]:
df_test = pd.read_csv("./test.csv", index_col=0)
df_test = data_pre(df_test)
predict = model.predict(df_test)
df_test["取引価格（総額）_log"] = predict
df_test[["取引価格（総額）_log"]].to_csv("submit_test.csv")

In [26]:


# パラメータ設定の例
params = {
    'objective': 'regression',
    'metric': 'mae',
    'boosting_type': 'gbdt',
    'learning_rate': 0.01,
    'num_leaves': 31,
    'max_depth': -1,
    'min_data_in_leaf': 20,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'lambda_l1': 0.1,
    'lambda_l2': 0.1,
    'min_split_gain': 0.01,
    'n_jobs': -1,
    'verbosity': -1,
    'device': 'gpu',
    'gpu_platform_id': 0,
    'gpu_device_id': 0,
    'max_bin': 25 ,
     'bin_construct_sample_cnt': 200000# max_binの値を255以下に設定
}


# データセットとモデルの設定
trains = lgb.Dataset(train_x, train_y)
valids = [lgb.Dataset(val_x, val_y)]

# モデルのトレーニング
model = lgb.train(
    params,
    trains,
    valid_sets=valids,
    num_boost_round=5000,  # より多くのブースティングラウンド
    # early_stopping_rounds=100,
    # verbose_eval=100
)

# 予測
predictions = model.predict(test_data, num_iteration=model.best_iteration)



LightGBMError: bin size 8341 cannot run on GPU